In [ ]:
import warnings

warnings.filterwarnings("ignore")
import logging

logging.disable(logging.CRITICAL)

import torch
import numpy as np
import pandas as pd
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from darts.metrics import mae
from darts.models import TiDEModel, TSMixerModel, DLinearModel
from darts.utils.likelihood_models import QuantileRegression
from darts.utils.callbacks import TFMProgressBar
from darts import TimeSeries
from src.CausalVAR.fitting import *

In [ ]:
df = pd.read_csv('../../data/census_shifted.csv')

In [ ]:
forecast_horizon = 1

In [ ]:
# Convert 'Year' to datetime
df['Year'] = pd.to_datetime(df['Year'])

# Calculate the total population for each 'Name' in 1991
df_1991 = df[df['Year'].dt.year == 1991].copy()
df_1991['Total Population 1991'] = df_1991['Population 0-14'] + df_1991['Population 15-64'] + df_1991['Population 65+']

# Create a dictionary to map 'Name' to 'Total Population 1991'
population_1991_dict = df_1991.set_index('Name')['Total Population 1991'].to_dict()

# Scale the specified columns for all years for each 'Name'
columns_to_scale = ['Population 0-14', 'Population 15-64', 'Population 65+', 'Births', 'Deaths', 'Migrations']
for col in columns_to_scale:
    df[col] = df.apply(lambda row: row[col] / population_1991_dict[row['Name']], axis=1)

# Display the first few rows of the modified dataframe
df.head()

In [ ]:
column_names = list(df.iloc[:,2:].columns)
df[column_names]=df[column_names].astype('float32')
'''series_names = {}
for name in list(df['Name'].unique()):
    series_names[name] = TimeSeries.from_dataframe(df[df['Name']==name],time_col='Year',value_cols=column_names).astype('float32')'''
series = TimeSeries.from_group_dataframe(df,group_cols='Name',time_col='Year',value_cols=column_names)


In [ ]:
train = []
val = []
test = [] 
for state in series:
    train_t, temp = state.split_after(0.5)
    val_t, test_t = temp.split_after(0.5)
    train.append(train_t)
    val.append(val_t)
    test.append(test_t)

In [ ]:
def create_params(
    input_chunk_length: int,
    output_chunk_length: int,
    full_training=True,
):
    # early stopping: this setting stops training once the the validation
    # loss has not decreased by more than 1e-5 for 10 epochs
    early_stopper = EarlyStopping(
        monitor="val_loss",
        patience=10,
        min_delta=1e-5,
        mode="min",
    )

    # PyTorch Lightning Trainer arguments (you can add any custom callback)
    if full_training:
        limit_train_batches = None
        limit_val_batches = None
        max_epochs = 200
        batch_size = 256
    else:
        limit_train_batches = 20
        limit_val_batches = 10
        max_epochs = 40
        batch_size = 64

    # only show the training and prediction progress bars
    progress_bar = TFMProgressBar(
        enable_sanity_check_bar=False, enable_train_bar=False, enable_validation_bar=False
    )
    pl_trainer_kwargs = {
        "gradient_clip_val": 1,
        "max_epochs": max_epochs,
        "limit_train_batches": limit_train_batches,
        "limit_val_batches": limit_val_batches,
        "accelerator": "auto",
        "callbacks": [early_stopper, progress_bar],
    }

    # optimizer setup, uses Adam by default
    optimizer_cls = torch.optim.Adam
    optimizer_kwargs = {
        "lr": 1e-4,
    }

    # learning rate scheduler
    lr_scheduler_cls = torch.optim.lr_scheduler.ExponentialLR
    lr_scheduler_kwargs = {"gamma": 0.999}

    # for probabilistic models, we use quantile regression, and set `loss_fn` to `None`
    likelihood = QuantileRegression()
    loss_fn = None

    return {
        "input_chunk_length": input_chunk_length,  # lookback window
        "output_chunk_length": output_chunk_length,  # forecast/lookahead window
        "use_reversible_instance_norm": True,
        "optimizer_kwargs": optimizer_kwargs,
        "pl_trainer_kwargs": pl_trainer_kwargs,
        "lr_scheduler_cls": lr_scheduler_cls,
        "lr_scheduler_kwargs": lr_scheduler_kwargs,
        "likelihood": likelihood,  # use a `likelihood` for probabilistic forecasts
        "loss_fn": loss_fn,  # use a `loss_fn` for determinsitic model
        "save_checkpoints": True,  # checkpoint to retrieve the best performing model state,
        "force_reset": True,
        "batch_size": batch_size,
        "random_state": 42,
        "add_encoders": {
            "cyclic": {
                "future": ["hour", "dayofweek", "month"]
            }  # add cyclic time axis encodings as future covariates
        },
    }

In [ ]:
input_chunk_length = 4
output_chunk_length = 1
use_static_covariates = False
full_training = True

In [ ]:
from torch.nn import MSELoss
def create_params2(input_chunk_length, output_chunk_length, full_training=True):
    early_stopper = EarlyStopping(
        monitor="val_loss",
        patience=10,
        min_delta=1e-5,
        mode="min",
    )

    if full_training:
        limit_train_batches = None
        limit_val_batches = None
        max_epochs = 200
        batch_size = 256
    else:
        limit_train_batches = 20
        limit_val_batches = 10
        max_epochs = 40
        batch_size = 64

    progress_bar = TFMProgressBar(
        enable_sanity_check_bar=False, enable_validation_bar=False
    )
    pl_trainer_kwargs = {
        "gradient_clip_val": 1,
        "max_epochs": max_epochs,
        "limit_train_batches": limit_train_batches,
        "limit_val_batches": limit_val_batches,
        "accelerator": "auto",
        "callbacks": [early_stopper, progress_bar],
    }

    optimizer_cls = torch.optim.Adam
    optimizer_kwargs = {
        "lr": 1e-4,
    }


    return {
        "input_chunk_length": input_chunk_length,
        "output_chunk_length": output_chunk_length,
        "use_reversible_instance_norm": True,
        "optimizer_kwargs": optimizer_kwargs,
        "pl_trainer_kwargs": pl_trainer_kwargs,
        "lr_scheduler_cls": torch.optim.lr_scheduler.ExponentialLR,
        "lr_scheduler_kwargs": {"gamma": 0.999},
        "loss_fn": MSELoss(),
        "save_checkpoints": True,
        "force_reset": True,
        "batch_size": batch_size,
        "random_state": 42,
        "add_encoders": {
            "cyclic": {
                "future": ["hour", "dayofweek", "month"]
            }
        },
    }

In [ ]:
# create the models
model_tsm = TSMixerModel(
    **create_params(
        input_chunk_length,
        output_chunk_length,
        full_training=full_training,
    ),
    use_static_covariates=use_static_covariates,
    model_name="tsm",
)
model_tide = TiDEModel(
    **create_params(
        input_chunk_length,
        output_chunk_length,
        full_training=full_training,
    ),
    use_static_covariates=use_static_covariates,
    model_name="tide",
)
model_dlinear = DLinearModel(
    **create_params2(
        input_chunk_length,
        output_chunk_length=1,
        full_training=full_training,
    ),
    use_static_covariates=use_static_covariates,
    model_name="dlinear",
)
models = {
    "TSM": model_tsm,
    "TiDE": model_tide, 
    "Dlinear": model_dlinear
}

In [ ]:
for model_name, model in models.items():
    model.fit(
        series=train,
        val_series=val,
    )
    # load from checkpoint returns a new model object, we store it in the models dict
    models[model_name] = model.load_from_checkpoint(
        model_name=model.model_name, best=True
    )

In [ ]:
forecast_horizon = 1

In [ ]:
# Perform backtesting with a rolling window
predictions_tide = models['TiDE'].historical_forecasts(
    series,
    start=0,
    forecast_horizon=forecast_horizon,
    stride=1,
    retrain=False,
    last_points_only=True,
    num_samples=1
)
for i in range(len(predictions_tide)):
    predictions_tide[i] = predictions_tide[i].slice_intersect(test[0])

predictions_tsm = models['TSM'].historical_forecasts(
    series,
    start=0,
    forecast_horizon=forecast_horizon,
    stride=1,
    retrain=False,
    last_points_only=True,
    num_samples=1
)
for i in range(len(predictions_tsm)):
    predictions_tsm[i] = predictions_tsm[i].slice_intersect(test[0])
    
predictions_dlinear = models['Dlinear'].historical_forecasts(
    series,
    start=0,
    forecast_horizon=forecast_horizon,
    stride=1,
    retrain=False,
    last_points_only=True,
    num_samples=1
)
time_index_series = predictions_dlinear[0].time_index
for i in range(len(predictions_dlinear)):
    predictions_dlinear[i] = predictions_dlinear[i].slice_intersect(test[0])

In [ ]:
from statsmodels.tsa.vector_ar.var_model import VAR
from SVAR.fitting_var_bounded import estimate_bounded
df1 = pd.read_csv('census_shifted.csv')
df_year = df1["Year"]
census = df1.drop(["Year"], axis=1)

n_states = census['Name'].nunique()
states = census['Name'].unique()

rows = census['Name'].value_counts().sort_index()
census.drop(['Name'], axis=1, inplace=True)
cols = ['Births', 'Deaths', 'Migrations', 'Population 0-14', 'Population 15-64', 'Population 65+']
census = census[cols]

# FITTING PHASE
df_array = census.to_numpy().reshape(n_states, -1, 6)
res_states = {}

def scale_data_by_pop(data):
    # Step 1: Calculate the sum of the last three columns at time n=0
    sum_last_three_cols = np.sum(data[:, 0, -3:], axis=1)
    # Step 2: Reshape to make the division broadcastable
    sum_last_three_cols = sum_last_three_cols[:, np.newaxis, np.newaxis]
    # Step 3: Divide each trajectory by the sum
    return data / sum_last_three_cols

def scale_data_by_pop_single_state(data):
    # Step 1: Calculate the sum of the last three columns at time n=0
    sum_last_three_cols = np.sum(data[0, -3:])
    # Step 2: Divide the entire trajectory by this sum
    return data / sum_last_three_cols

df_array = scale_data_by_pop(df_array)

predictions_var = []


def get_forecast(process, past_data, future_steps):
    forecasts = []
    for i in range(past_data.shape[0]):
        forecast_trajectory = process.forecast(past_data[i], steps=future_steps)
        forecasts.append(forecast_trajectory)
    return np.array(forecasts)

for state in range(n_states):
    model = VAR(df_array[state, :, :])
    lag_order = 1
    res = estimate_bounded(model=model, lags=1, trend="c")
    tensor_lagged = to_lagged_tensor(df_array[state, :, :].reshape(1, -1, 6), lags=1)
    tensor_lagged = tensor_lagged[0, :-1, :, :].transpose(1, 0, 2)
    if forecast_horizon == 1:
        prediction_state = get_forecast(res, tensor_lagged, forecast_horizon)[:, -1, :].reshape(-1, 6)
        series_prediction_state = TimeSeries.from_times_and_values(series[0].time_index[1:], prediction_state, columns=cols)
    else:
        prediction_state = get_forecast(res, tensor_lagged, forecast_horizon)[input_chunk_length-1:-forecast_horizon+1, -1, :].reshape(-1, 6)
        series_prediction_state = TimeSeries.from_times_and_values(time_index_series, prediction_state, columns=cols)
    predictions_var.append(series_prediction_state)

In [ ]:
for i in range(len(predictions_var)):
    predictions_var[i] = predictions_var[i].slice_intersect(test[0])

In [ ]:
check_cols = ['Population 0-14', 'Population 15-64', 'Population 65+']
print(f'Forecast Horizon: {forecast_horizon}')
mae_var = []
for i in range(len(test)):
    for col in check_cols:
        mae_var.append(mae(test[i][col], predictions_var[i][col]))
print(f'var {np.mean(mae_var)}')


mae_dlinear = []
for i in range(len(test)):
    for col in check_cols:
        mae_dlinear.append(mae(test[i][col], predictions_dlinear[i][col]))
print(f'dlinear {np.mean(mae_dlinear)}')

mae_tsm = []
for i in range(len(test)):
    for col in check_cols:
        mae_tsm.append(mae(test[i][col], predictions_tsm[i][col]))
print(f'tsm {np.mean(mae_tsm)}')

mae_tide = []
for i in range(len(test)):
    for col in check_cols:
        mae_tide.append(mae(test[i][col], predictions_tide[i][col]))
print(f'tide {np.mean(mae_tide)}')

In [ ]:


def calculate_and_print_metrics():
    check_cols = ['Population 0-14', 'Population 15-64', 'Population 65+']
    metrics = [mae, smape, rmse]
    metric_names = ['MAE', 'SMAPE', 'RMSE']

    for metric, metric_name in zip(metrics, metric_names):
        print(f'\n{metric_name}:')
        print(f'Forecast Horizon: {forecast_horizon}')

        # Calculate metrics for each model
        metric_var = []
        for i in range(len(test)):
            for col in check_cols:
                metric_var.append(metric(test[i][col], predictions_var[i][col]))
        print(f'var {np.mean(metric_var)}')

        metric_dlinear = []
        for i in range(len(test)):
            for col in check_cols:
                metric_dlinear.append(metric(test[i][col], predictions_dlinear[i][col]))
        print(f'dlinear {np.mean(metric_dlinear)}')

        metric_tsm = []
        for i in range(len(test)):
            for col in check_cols:
                metric_tsm.append(metric(test[i][col], predictions_tsm[i][col]))
        print(f'tsm {np.mean(metric_tsm)}')

        metric_tide = []
        for i in range(len(test)):
            for col in check_cols:
                metric_tide.append(metric(test[i][col], predictions_tide[i][col]))
        print(f'tide {np.mean(metric_tide)}')

        print()  # Add a blank line between metrics

calculate_and_print_metrics()

In [ ]:
import numpy as np
from darts.metrics import smape, rmse, mae


def calculate_and_print_metrics():
    check_cols = ['Population 0-14', 'Population 15-64', 'Population 65+']
    metrics = [mae, smape, rmse]
    metric_names = ['MAE', 'SMAPE', 'RMSE']

    for metric, metric_name in zip(metrics, metric_names):
        print(f'\n{metric_name}:')
        print(f'Forecast Horizon: {forecast_horizon}')

        # Calculate metrics for each model
        metric_var = []
        for i in range(len(test)):
            for col in check_cols:
                metric_var.append(metric(test[i][col], predictions_var[i][col]))
        print(f'var {np.mean(metric_var)}')

        metric_dlinear = []
        for i in range(len(test)):
            for col in check_cols:
                metric_dlinear.append(metric(test[i][col], predictions_dlinear[i][col]))
        print(f'dlinear {np.mean(metric_dlinear)}')

        metric_tsm = []
        for i in range(len(test)):
            for col in check_cols:
                metric_tsm.append(metric(test[i][col], predictions_tsm[i][col]))
        print(f'tsm {np.mean(metric_tsm)}')

        metric_tide = []
        for i in range(len(test)):
            for col in check_cols:
                metric_tide.append(metric(test[i][col], predictions_tide[i][col]))
        print(f'tide {np.mean(metric_tide)}')

        print()  # Add a blank line between metrics

calculate_and_print_metrics()